In [1]:
# AQI Calculation for Vietnam's Provinces

## Step 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
## Step 2: Load Data
# Assuming you have data for each province in a CSV format with columns like 'Province', 'Timestamp', 'PM2.5', 'PM10', 'O3', etc.
data = pd.read_csv('air_quality/air_quality.csv', index_col = 0, header = [0,1])  # Replace with your actual data path
#data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data.head()

Ho Chi Minh City, Vietnam                              \
                                           co    no2   o3     so2   pm2_5   
time                                                                        
2020-11-28 00:00:00                   8438.11  74.71  0.0  105.86  328.59   
2020-11-28 01:00:00                   5393.98  57.58  0.0   58.17  209.73   
2020-11-28 02:00:00                   1789.09  38.04  0.1   17.17   66.01   
2020-11-28 03:00:00                    814.44  26.39  2.3    9.54   29.97   
2020-11-28 04:00:00                    547.41  19.19  8.4    8.46   20.61   

                                 Hanoi, Vietnam                ...  \
                       pm10  aqi             co    no2     o3  ...   
time                                                           ...   
2020-11-28 00:00:00  477.11  5.0         814.44  30.16  11.80  ...   
2020-11-28 01:00:00  307.78  5.0         720.98  25.02  14.48  ...   
2020-11-28 02:00:00   97.71  5.0         680.92  22.96  14.66  ...   
2020-11-28 03:00:00   46.00  4.0         687.60  23.99  12.16  ...   
2020-11-28 04:00:00   32.66  3.0         700.95  25.71   9.30  ...   

                    Kon Tum, Vietnam             Đà Nẵng, Vietnam        \
                               pm2_5   pm10  aqi               co   no2   
time                                                                      
2020-11-28 00:00:00            12.45  14.65  2.0           574.11  4.88   
2020-11-28 01:00:00            13.51  15.16  2.0           560.76  3.34   
2020-11-28 02:00:00            15.60  17.06  2.0           534.06  2.64   
2020-11-28 03:00:00            17.97  19.39  2.0           520.71  2.40   
2020-11-28 04:00:00            20.57  21.97  3.0           527.38  2.38   

                                                       
                         o3    so2  pm2_5   pm10  aqi  
time                                                   
2020-11-28 00:00:00   98.71   9.78  74.42  96.75  5.0  
2020-11-28 01:00:00  104.43   9.30  73.33  95.85  5.0  
2020-11-28 02:00:00  105.86   9.06  69.25  90.21  5.0  
2020-11-28 03:00:00  101.57   9.54  65.85  85.18  5.0  
2020-11-28 04:00:00   95.84  10.73  65.12  82.40  5.0  

[5 rows x 441 columns]

In [3]:
data.columns

MultiIndex([('Ho Chi Minh City, Vietnam',    'co'),
            ('Ho Chi Minh City, Vietnam',   'no2'),
            ('Ho Chi Minh City, Vietnam',    'o3'),
            ('Ho Chi Minh City, Vietnam',   'so2'),
            ('Ho Chi Minh City, Vietnam', 'pm2_5'),
            ('Ho Chi Minh City, Vietnam',  'pm10'),
            ('Ho Chi Minh City, Vietnam',   'aqi'),
            (           'Hanoi, Vietnam',    'co'),
            (           'Hanoi, Vietnam',   'no2'),
            (           'Hanoi, Vietnam',    'o3'),
            ...
            (         'Kon Tum, Vietnam', 'pm2_5'),
            (         'Kon Tum, Vietnam',  'pm10'),
            (         'Kon Tum, Vietnam',   'aqi'),
            (         'Đà Nẵng, Vietnam',    'co'),
            (         'Đà Nẵng, Vietnam',   'no2'),
            (         'Đà Nẵng, Vietnam',    'o3'),
            (         'Đà Nẵng, Vietnam',   'so2'),
            (         'Đà Nẵng, Vietnam', 'pm2_5'),
            (         'Đà Nẵng, Vietnam',  'pm10

In [4]:
# Define breakpoints for AQI calculation
breakpoints = {
    'pm2_5': [(0, 35, 0, 50), (36, 75, 51, 100), (76, 115, 101, 150), (116, 150, 151, 200), 
              (151, 250, 201, 300), (251, 350, 301, 400), (351, 500, 401, 500)],
    'pm10': [(0, 50, 0, 50), (51, 150, 51, 100), (151, 250, 101, 150), (251, 350, 151, 200), 
             (351, 420, 201, 300), (421, 500, 301, 400), (501, 600, 401, 500)],
    'o3': [(0, 160, 0, 50), (161, 200, 51, 100), (201, 300, 101, 150), (301, 400, 151, 200), 
           (401, 800, 201, 300), (801, 1000, 301, 400), (1001, 1200, 401, 500)],
    'co': [(0, 5000, 0, 50), (5001, 10000, 51, 100), (10001, 35000, 101, 150), (35001, 60000, 151, 200), 
           (60001, 90000, 201, 300), (90001, 120000, 301, 400), (120001, 150000, 401, 500)],
    'no2': [(0, 100, 0, 50), (101, 200, 51, 100), (201, 700, 101, 150), (701, 1200, 151, 200), 
            (1201, 2340, 201, 300), (2341, 3090, 301, 400), (3091, 3840, 401, 500)],
    'so2': [(0, 150, 0, 50), (151, 500, 51, 100), (501, 650, 101, 150), (651, 800, 151, 200), 
            (801, 1600, 201, 300), (1601, 2100, 301, 400), (2101, 2620, 401, 500)]
}

In [5]:
def calculate_aqi(concentration, pollutant):
    if pd.notna(concentration):
        if pollutant in breakpoints:
            for (C_low, C_high, I_low, I_high) in breakpoints[pollutant]:
                if C_low <= concentration <= C_high:
                    return ((I_high - I_low) / (C_high - C_low)) * (concentration - C_low) + I_low
    return None

# Function to calculate AQI for a row with MultiIndex columns
def calculate_aqi_row(row, pollutants, province):
    aqi_values = []  # List to hold AQI values
    for p in pollutants:
        concentration = row.get((province, p))  # Get the concentration for this pollutant
        if pd.notna(concentration):  # Only calculate AQI if concentration is valid
            aqi = calculate_aqi(concentration, p)
            if aqi is not None:  # Only add valid AQI values
                aqi_values.append(aqi)
    
    # Return the maximum AQI value, or None if there are no valid AQI values
    return max(aqi_values, default=None)


In [9]:
if 'time' in data.columns.get_level_values(0):
    time_data = data[('time', '')].copy()  # Copy 'time' column
    data = data.drop(columns=[('time', '')])  # Remove 'time' column for processing
else:
    time_data = None  # No time column available

# Get unique provinces and pollutants
provinces = data.columns.get_level_values(0).unique()
pollutants = ['pm2_5', 'pm10', 'o3', 'co', 'no2', 'so2']

# Calculate AQI for each province
for province in provinces:
    if province not in ['time', '']:  # Skip 'time' and unnamed columns
        data[(province, 'AQI')] = data.apply(calculate_aqi_row, axis=1, args=(pollutants, province))
if time_data is not None:
    data.insert(0, ('time', ''), time_data)  # Insert 'time' as the first column
# Save the updated DataFrame
data.to_csv('air_quality/air_quality.csv', index=False)


In [10]:
updated_df = pd.read_csv('air_quality/air_quality.csv', index_col = 0, header = [0,1])
updated_df.head()

Ho Chi Minh City, Vietnam Ho Chi Minh City, Vietnam                       \
co                                              no2   o3     so2   pm2_5   
8438.11                                       74.71  0.0  105.86  328.59   
5393.98                                       57.58  0.0   58.17  209.73   
1789.09                                       38.04  0.1   17.17   66.01   
814.44                                        26.39  2.3    9.54   29.97   
547.41                                        19.19  8.4    8.46   20.61   

Ho Chi Minh City, Vietnam              Hanoi, Vietnam                       \
co                           pm10  aqi             co    no2     o3    so2   
8438.11                    477.11  5.0         814.44  30.16  11.80  22.89   
5393.98                    307.78  5.0         720.98  25.02  14.48  19.07   
1789.09                     97.71  5.0         680.92  22.96  14.66  17.41   
814.44                      46.00  4.0         687.60  23.99  12.16  18.12   
547.41                      32.66  3.0         700.95  25.71   9.30  19.79   

Ho Chi Minh City, Vietnam  ... Vị Thanh, Vietnam Đông Hà, Vietnam  \
co                         ...               AQI              AQI   
8438.11                    ...         42.770000       205.410000   
5393.98                    ...         49.370000       207.950000   
1789.09                    ...         55.108462       178.512059   
814.44                     ...         61.578974       131.983077   
547.41                     ...         64.556667       117.823333   

Ho Chi Minh City, Vietnam Cao Bằng, Vietnam Điện Biên Phủ, Vietnam  \
co                                      AQI                    AQI   
8438.11                           21.471429              95.489487   
5393.98                           22.085714              96.130256   
1789.09                           22.428571              96.695641   
814.44                            22.671429              96.142821   
547.41                            22.957143              94.459231   

Ho Chi Minh City, Vietnam Gia Nghĩa, Vietnam Hà Giang, Vietnam  \
co                                       AQI               AQI   
8438.11                            20.340625         16.457143   
5393.98                            19.668750         15.942857   
1789.09                            19.893750         15.100000   
814.44                             20.340625         14.057143   
547.41                             20.562500         13.242857   

Ho Chi Minh City, Vietnam Bắc Kạn, Vietnam Lai Châu, Vietnam Kon Tum, Vietnam  \
co                                     AQI               AQI              AQI   
8438.11                          19.285714         52.394615        17.785714   
5393.98                          20.414286         58.136410        19.300000   
1789.09                          21.614286         62.860513        22.285714   
814.44                           22.571429         67.546923        25.671429   
547.41                           23.271429         70.914103        29.385714   

Ho Chi Minh City, Vietnam Đà Nẵng, Vietnam  
co                                     AQI  
8438.11                          99.271282  
5393.98                          97.901795  
1789.09                          92.775641  
814.44                           88.503846  
547.41                           87.586667  

[5 rows x 503 columns]

In [11]:
## Step 5: Visualize Results (Optional)
# Example: Plot daily AQI for a specific province
province = 'Hanoi'  # Replace with any province you want to visualize
province_daily_aqi = daily_aqi[daily_aqi['Province'] == province]

plt.figure(figsize=(10, 5))
plt.plot(province_daily_aqi['Day'], province_daily_aqi['AQI'], label=f'Daily AQI - {province}')
plt.xlabel('Date')
plt.ylabel('AQI')
plt.title(f'Daily AQI for {province}')
plt.legend()
plt.show()


NameError: name 'daily_aqi' is not defined